In [ ]:
# dependencies
import kagglehub
from kagglehub import KaggleDatasetAdapter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf
import keras_tuner
import os
from sklearn.metrics import classification_report, f1_score

In [6]:
# Set the path to the file you'd like to load
file_path = "products.csv"

# Load the latest version
df = kagglehub.dataset_load(
    KaggleDatasetAdapter.PANDAS,
    "poorveshchaudhari/amazon-fashion-products",
    file_path
)

100%|██████████| 2.64M/2.64M [00:00<00:00, 5.71MB/s]


In [7]:
df = df.dropna(subset=["image_url", "brand"])
df = df[["image_url", "brand"]]

# since there are brands with ONLY 1 record, group them into the misc class
min_count = 10
brand_counts = df["brand"].value_counts()
df["brand"] = df["brand"].apply(lambda x: x if brand_counts[x] >= min_count else "misc")

# now cast the classes into integers.
label_encoder = LabelEncoder()
df["brand"] = label_encoder.fit_transform(df["brand"])
df["brand"] = df["brand"].astype(np.int32)

train_df, test_df = train_test_split(df, test_size = 0.2, random_state = 42)
train_df, val_df = train_test_split(train_df, test_size = 0.2, random_state = 42)


In [8]:
# prepare the data
IMG_DIR = "images"
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

def fetch_images(path, label):
    path = path.numpy().decode('utf-8')

    try:
        img = tf.keras.utils.load_img(path, target_size = IMG_SIZE)
        img = tf.keras.utils.img_to_array(img) / 255.0 # since every pixel has values [0, 255], dividing by 255 normalizes them to [0, 1]
    except Exception:
        img = np.zeros((*IMG_SIZE, 3), dtype = np.float32)

    return img.astype(np.float32), np.int32(label)

def tf_fetch_image(url, label):
    img, lbl = tf.py_function(fetch_images, [url, label], [tf.float32, tf.int32])
    img.set_shape((*IMG_SIZE, 3))
    lbl.set_shape(())

    return img, lbl

def make_datasets(sub_dataframe):
    paths = [os.path.join(IMG_DIR, f"{i}.jpg") for i in sub_dataframe.index]

    ds = tf.data.Dataset.from_tensor_slices((paths, sub_dataframe["brand"].values))
    ds = ds.map(tf_fetch_image, num_parallel_calls = tf.data.AUTOTUNE)
    ds = ds.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return ds

train_ds, val_ds, test_ds = make_datasets(train_df), make_datasets(val_df), make_datasets(test_df)


print(train_ds)


<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [9]:
# here we can visualize the data (optional)

"""
the first for loop displays:

(32, 224, 224, 3) [3.3, ...., 1.5]

Here we have:
- a batch of 32 images
- 224x224 each (and their 3 respetive channels)
- [ints] the array of brands for each image
"""

for img, lbl in train_ds.take(1):
    print(img.shape, lbl.numpy())

"""
second for loop shows the values for each pixel

For each image we have:
- first axis: rows (height = 224)
- second axis: columns (width = 224)
- third axis: channels (RGB = 3)
"""
for imgs, labels in train_ds.take(1):
    print("Image shape:", imgs[1].shape)
    print("Pixel values (first image):")
    print(imgs[1].numpy())


(32, 128, 128, 3) [113 144  62 149 173  81 124 184  30  46 113 184  48 113  92 140 182  40
 184 184   0  48  97 184  11 184  15 184  67 143  12  42]
Image shape: (128, 128, 3)
Pixel values (first image):
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 ...

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]


In [ ]:
def build_cnn_model(hp, num_classes):
    model = tf.keras.Sequential()

    # input layer
    model.add(tf.keras.layers.Input(shape = (128, 128, 3)))

    # convolutional layers
    for i in range(hp.Int("num_conv_layers", 1, 3)):
        model.add(tf.keras.layers.Conv2D(
            filters = hp.Int(f"filters_{i}", min_value = 32, max_value = 128, step = 32),
            kernel_size = hp.Choice("kernel_size", values = [3, 5]),
            activation = "relu",
            padding = "same"
        ))
        model.add(tf.keras.layers.MaxPooling2D(pool_size = 2))

    model.add(tf.keras.layers.av())

    # dense layers
    for j in range(hp.Int("num_dense_layers", 0, 2)):
        model.add(tf.keras.layers.Dense(
            units = hp.Int(f"units_dense_{j}", min_value = 64, max_value = 256, step = 64),
            activation = hp.Choice("activation", ["relu", "tanh"])
        ))

    # output layer for classification
    model.add(tf.keras.layers.Dense(num_classes, activation = "softmax"))

    # hyperparameter: learning rate
    hp_lr = hp.Choice("learning_rate", values = [1e-2, 1e-3, 1e-4])

    # compile for classification
    model.compile(
        optimizer = tf.keras.optimizers.Adam(learning_rate = hp_lr),
        loss = "sparse_categorical_crossentropy",
        metrics = ["accuracy"]
    )

    return model

In [ ]:
build_cnn_model(keras_tuner.HyperParameters())

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel = build_cnn_model,
    objective = "val_accuracy",
    max_trials = 10,
    directory = "my_dir",
    project_name = "cnn_classification_tunning",
    overwrite = True
)

tuner.search_space_summary()

In [ ]:
tuner.search(
    train_ds,
    validation_data = val_ds,
    epochs = 5,
)

In [ ]:
tuner.results_summary()

In [ ]:
# extract the best model
best_model = tuner.get_best_models(num_models = 1)[0]
best_model.summary()

In [ ]:
# metrics

y_true = np.concatenate([y for x, y in test_ds], axis = 0)
y_pred_probs = best_model.predict(test_ds)
y_pred = np.argmax(y_pred_probs, axis=1)
weighted_f1 = f1_score(y_true, y_pred, average='weighted')
loss, acc = best_model.evaluate(test_ds)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {acc:.4f}")
print(f"Weighted F1: {weighted_f1:.4f}")